In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
from time import sleep
import time
import os
country_codes = {'Belgium':'19','Austria':'127','Denmark':'39','Netherlands':'122','Portugal':'136',
                 'Switzerland':'148'
                 }

def dict_to_list(d, pair_sep=":", value_sep=","):
    result = []
    for key, values in d.items():
        if isinstance(values, list):
            value_str = value_sep.join(map(str, values))
        else:
            value_str = str(values)
        result.append(f"{key}{pair_sep}{value_str}")
    return result

out_folder = 'datasets/'


In [2]:
# get top 1 leagues for a country
def get_top_leagues(country_id: str):
    country_id2 = country_codes[country_id]
    url = f"https://www.transfermarkt.com/wettbewerbe/national/wettbewerbe/{country_id2}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    table = soup.find("table", class_="items")
    rows = table.find("tbody").find_all("tr") if table else []
    
    competitions = []
    for row in rows:
        if not (row.has_attr("class") and ("odd" in row["class"] or "even" in row["class"])):
            continue
        main_td = row.find("td", class_="hauptlink")
        if not main_td:
            continue
        nested_table = main_td.find("table", class_="inline-table")
        if not nested_table:
            continue
        nested_cells = nested_table.find_all("td")
        if len(nested_cells) < 2:
            continue
        link_tag = nested_cells[1].find("a")
        if not link_tag:
            continue
        league_name = link_tag.get("title", "").strip()
        href = link_tag["href"]
        competition_id = href.split("/wettbewerb/")[1].split("/")[0]
        competitions.append({
            "league_name": league_name,
            "competition_id": competition_id
        })
    # assign tier by order
    for idx, comp in enumerate(competitions):
        comp['tier'] = idx + 1


    df_competitions = pd.DataFrame(competitions[:1])
    df_competitions['country_name'] = country_id
    df_competitions['country_id'] = country_id2
    return df_competitions
   

# get last 5 seasons
def get_last_10_seasons(competition_id):
    url = f"https://www.transfermarkt.com/{competition_id}/transfers/wettbewerb/{competition_id}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    select = soup.find("select", {"name": "saison_id"})
    if not select:
        return []
    seasons = []
    for option in select.find_all("option"):
        try:
            val = int(option['value'])
            seasons.append(val)
        except:
            continue
    
    sorted(seasons, reverse=True)[:5]

    seasons_df = pd.DataFrame(sorted(seasons, reverse=True)[:5])
    seasons_df['competition_id'] = competition_id
    
    return seasons_df

In [3]:
## get all seasons for top 1 tiers per country
if os.path.exists(out_folder+'transfermarkt_seasons_final.csv'):
    allSeasons = pd.read_csv(out_folder+'transfermarkt_seasons_final.csv',sep=',')
else:
    leagues_list = []
    for l in country_codes:
        leagues = get_top_leagues(l)
        leagues_list.append(leagues)
    
    allLeagues = pd.concat(leagues_list)
    
    seasons_list = []
    for c in allLeagues.competition_id:
        seasons = get_last_10_seasons(c)
        seasons['country'] = l
        seasons_list.append(seasons)
        print(len(seasons_list))


    allSeasons = pd.concat(seasons_list)
    allSeasons = allSeasons.rename(columns={0:'season_start_year'})



print(len(allSeasons))
allSeasons.to_csv(out_folder+'transfermarkt_seasons_final.csv',mode='w',index=False,sep=',')

1
2
3
4
5
6
30


In [21]:
comp_names = {'DK1':'superligaen','BE1':'jupiler-pro-league','A1':'bundesliga','NL1':'eredivisie','PO1':'liga-nos','C1':'super-league'}
allSeasons['competition_name'] = allSeasons['competition_id'].map(comp_names)

In [28]:
def get_season_team_urls(start_year, comp,comp_name):
    # URL of the league table for season and competition
    headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0 Safari/537.36"
    )
    }

    league_url = f"https://www.transfermarkt.com/{comp_name}/startseite/wettbewerb/{comp}/plus/?saison_id={start_year}"

    # Fetch page
    response = requests.get(league_url, headers=headers)
    print(response.status_code)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find team links in the standings table
    team_links = []
    for a in soup.select('table.items td.hauptlink a'):
        href = a.get("href")
        if href and "/verein/" in href:  # filter only team URLs
            team_links.append("https://www.transfermarkt.com" + href)

    # Remove duplicates (optional)
    team_links = list(dict.fromkeys(team_links))
    team_links = [x for x in team_links if 'startseite' in x]

    team_links = pd.DataFrame(team_links,columns=['team_url'])
    team_links['season_start_year'] = start_year
    team_links['competition_id'] = comp
    
    return team_links




In [29]:
unique_rows = allSeasons[['competition_id','competition_name','season_start_year']].drop_duplicates().itertuples(index=False, name=None)
unique_rows = list(unique_rows)

In [30]:
unique_rows

[('BE1', 'jupiler-pro-league', 2027),
 ('BE1', 'jupiler-pro-league', 2026),
 ('BE1', 'jupiler-pro-league', 2025),
 ('BE1', 'jupiler-pro-league', 2024),
 ('BE1', 'jupiler-pro-league', 2023),
 ('A1', 'bundesliga', 2027),
 ('A1', 'bundesliga', 2026),
 ('A1', 'bundesliga', 2025),
 ('A1', 'bundesliga', 2024),
 ('A1', 'bundesliga', 2023),
 ('DK1', 'superligaen', 2027),
 ('DK1', 'superligaen', 2026),
 ('DK1', 'superligaen', 2025),
 ('DK1', 'superligaen', 2024),
 ('DK1', 'superligaen', 2023),
 ('NL1', 'eredivisie', 2027),
 ('NL1', 'eredivisie', 2026),
 ('NL1', 'eredivisie', 2025),
 ('NL1', 'eredivisie', 2024),
 ('NL1', 'eredivisie', 2023),
 ('PO1', 'liga-nos', 2027),
 ('PO1', 'liga-nos', 2026),
 ('PO1', 'liga-nos', 2025),
 ('PO1', 'liga-nos', 2024),
 ('PO1', 'liga-nos', 2023),
 ('C1', 'super-league', 2027),
 ('C1', 'super-league', 2026),
 ('C1', 'super-league', 2025),
 ('C1', 'super-league', 2024),
 ('C1', 'super-league', 2023)]

In [31]:
failed_list = []
team_links_list = []
for t in unique_rows:
    s = t[2]
    c = t[0]
    co = t[1]
    try:
        team_links = get_season_team_urls(s,c,co)
        team_links_list.append(team_links)
        team_links.to_csv(out_folder+'transfermarkt_teams_final.csv',mode='a',index=False,sep=',')
        sleep_num = random.uniform(4.67,19.3)
        sleep(sleep_num)
        print(f"slept for {round(sleep_num,2)} seconds.")
    except:
        failed_list.append((s,c))

allTeams = pd.concat(team_links_list)
print(len(allTeams))


    



200
slept for 16.99 seconds.
200
slept for 12.05 seconds.
200
slept for 10.27 seconds.
200
slept for 6.14 seconds.
200
slept for 16.52 seconds.
200
slept for 17.79 seconds.
200
slept for 6.4 seconds.
200
slept for 18.88 seconds.
200
slept for 7.94 seconds.
200
slept for 10.21 seconds.
200
slept for 14.46 seconds.
200
slept for 12.67 seconds.
200
slept for 11.36 seconds.
200
slept for 12.24 seconds.
503
slept for 11.5 seconds.
503
slept for 5.57 seconds.
503
slept for 9.82 seconds.
200
slept for 7.8 seconds.
503
slept for 15.0 seconds.
503
slept for 8.3 seconds.
503
slept for 5.29 seconds.
200
slept for 17.58 seconds.
200
slept for 5.29 seconds.
200
slept for 19.25 seconds.
200
slept for 7.48 seconds.
200
slept for 5.39 seconds.
200
slept for 12.3 seconds.
200
slept for 9.01 seconds.
200
slept for 4.86 seconds.
503
slept for 18.11 seconds.
326


In [35]:
def get_players_in_season(tu):
    # Team URL – detailed view with player links:
    url = tu
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    print(response.status_code)
    soup = BeautifulSoup(response.text, "html.parser")

    td = soup.find_all('td', class_ ='hauptlink')

    player_links = []
    for t in td:
        try:
            a = t.find('a', href = True)
            a = a['href']
            player_links.append(a)
        except:
            print(f"No player link for {t} in {tu}")

    player_links = [x for x in player_links if 'marktwert' in x]
    player_links = pd.DataFrame(player_links,columns=['player_url'])

    return player_links



In [ ]:
allTeams = pd.read_csv('datasets/transfermarkt_teams_final.csv',sep=',')
allTeams = allTeams[allTeams['team_url'] != 'team_url']

In [37]:
team_links = allTeams['team_url'].tolist()

allPlayers = pd.read_csv('datasets/transfermarkt_player_links.csv',sep=',')

team_links = [x for x in team_links if x not in allPlayers.team_link.unique().tolist()]

batch_list = [team_links[x:x+10] for x in range(0, len(team_links), 10)]
batch_list[1:3]

[['https://www.transfermarkt.com/fc-brugge/startseite/verein/2282/saison_id/2025',
  'https://www.transfermarkt.com/krc-genk/startseite/verein/1184/saison_id/2025',
  'https://www.transfermarkt.com/rsc-anderlecht/startseite/verein/58/saison_id/2025',
  'https://www.transfermarkt.com/standard-luttich/startseite/verein/3057/saison_id/2025',
  'https://www.transfermarkt.com/oud-heverlee-leuven/startseite/verein/2727/saison_id/2025',
  'https://www.transfermarkt.com/malmo-ff/startseite/verein/496/saison_id/2024',
  'https://www.transfermarkt.com/hammarby-if/startseite/verein/1059/saison_id/2024',
  'https://www.transfermarkt.com/djurgardens-if/startseite/verein/1044/saison_id/2024',
  'https://www.transfermarkt.com/if-elfsborg-boras/startseite/verein/1101/saison_id/2024',
  'https://www.transfermarkt.com/aik-solna/startseite/verein/272/saison_id/2024'],
 ['https://www.transfermarkt.com/bk-hacken/startseite/verein/1109/saison_id/2024',
  'https://www.transfermarkt.com/ifk-goteborg/startseit

In [38]:
from tqdm import tqdm
players_list = []
batch_counter = 1
for b in tqdm(range(len(batch_list))):
    for t in batch_list[b]:
        sleep(random.uniform(9.5,15.6))
        try:
            player_links = get_players_in_season(t)
            player_links['team_link'] = t
            players_list.append(player_links)
        except:
            print(f"failed for {t} in batch {batch_counter}")
    
    print(f"Batch {batch_counter} / {len(batch_list)} finished.")
    batch_counter += 1
    sleep(random.uniform(43,74.5))
    allPlayers = pd.concat(players_list)



allPlayers['player_id'] = allPlayers['player_url'].str.split('/').str[-1]
allPlayers.to_csv(out_folder+'transfermarkt_player_links2.csv',sep=',',index=False,mode='a')

  0%|          | 0/78 [00:00<?, ?it/s]

200
200
200
200
200
200
200
200
200
200
Batch 1 / 78 finished.


  1%|▏         | 1/78 [03:15<4:10:34, 195.26s/it]

200
200
200
200
200
200
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/malmo-ff/startseite/verein/496/saison_id/2024
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/malmo-ff/startseite/verein/496/saison_id/2024
200
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/hammarby-if/startseite/verein/1059/saison_id/2024
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/hammarby-if/startseite/verein/1059/saison_id/2024
200
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/djurgardens-if/startseite/verein/1044/saison_id/2024
200
200
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/aik-solna/startseite/verein/272/saison_id/2024
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/aik-solna/startseite/verein/272/saison_id/2024
No pl

  3%|▎         | 2/78 [06:42<4:16:20, 202.37s/it]

...

 97%|█████████▋| 76/78 [4:20:57<06:45, 202.94s/it]

200
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/minnesota-united-fc/startseite/verein/56089/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/minnesota-united-fc/startseite/verein/56089/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/minnesota-united-fc/startseite/verein/56089/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/minnesota-united-fc/startseite/verein/56089/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/minnesota-united-fc/startseite/verein/56089/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/minnesota-united-fc/startseite/verein/56089/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/minnesota-united-fc/starts

 99%|█████████▊| 77/78 [4:24:11<03:20, 200.27s/it]

200
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/st-louis-city-sc/startseite/verein/82686/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/st-louis-city-sc/startseite/verein/82686/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/st-louis-city-sc/startseite/verein/82686/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/st-louis-city-sc/startseite/verein/82686/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/st-louis-city-sc/startseite/verein/82686/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/st-louis-city-sc/startseite/verein/82686/saison_id/2023
No player link for <td class="rechts hauptlink">-</td> in https://www.transfermarkt.com/st-louis-city-sc/startseite/verein/82686/sai

100%|██████████| 78/78 [4:25:45<00:00, 204.43s/it]


In [41]:
## reload the allPlayers dataset with new info
allPlayers = pd.read_csv('datasets/transfermarkt_player_links2.csv',sep=',')
print(f"Info found on {len(allPlayers)} players.")
print(allPlayers.columns)
allPlayers = allPlayers.drop_duplicates(subset=['player_url'])
print(f"After dropping duplicate values, {len(allPlayers)} players remain.")

allPlayers


Info found on 38148 players.
Index(['player_url', 'team_link', 'player_id'], dtype='object')
After dropping duplicate values, 6885 players remain.


,player_url,team_link,player_id
0,/simon-mignolet/marktwertverlauf/spieler/50219,https://www.transfermarkt.com/fc-brugge/starts...,50219
1,/nordin-jackers/marktwertverlauf/spieler/250543,https://www.transfermarkt.com/fc-brugge/starts...,250543
2,/josef-bursik/marktwertverlauf/spieler/491737,https://www.transfermarkt.com/fc-brugge/starts...,491737
3,/dani-van-den-heuvel/marktwertverlauf/spieler/...,https://www.transfermarkt.com/fc-brugge/starts...,559320
4,/joel-ordonez/marktwertverlauf/spieler/893661,https://www.transfermarkt.com/fc-brugge/starts...,893661
...,...,...,...
32632,/leonardo-bertone/marktwertverlauf/spieler/194975,https://www.transfermarkt.com/fc-thun/startsei...,194975
32634,/franz-ethan-meichtry/marktwertverlauf/spieler...,https://www.transfermarkt.com/fc-thun/startsei...,1172166
32635,/marc-gutbub/marktwertverlauf/spieler/660514,https://www.transfermarkt.com/fc-thun/startsei...,660514
32637,/layton-stewart/marktwertverlauf/spieler/580801,https://www.transfermarkt.com/fc-thun/startsei...,580801


In [50]:
currentPlayerValues = pd.read_csv('datasets/transfermarkt_players_values_2.csv',sep=',')
print(f"Info found on {len(currentPlayerValues)} players.")
currentPlayerValues = currentPlayerValues[currentPlayerValues['playerId'] != 'playerId']
print(f"After dropping header rows, {len(currentPlayerValues)} players remain.")
current_ids = currentPlayerValues['playerId'].unique().tolist()
print(len(current_ids))

Info found on 16191 players.
After dropping header rows, 15995 players remain.
958


In [52]:
player_ids = [x for x in allPlayers['player_id'].unique().tolist() if x not in current_ids]
print(len(player_ids))
batch_players = [player_ids[x:x+30] for x in range(0, len(player_ids), 30)]
print(len(batch_players))

6565
219


In [ ]:

player_df_list = []
try_again = []
batch_counter = 1
for b in batch_players:
    for id in b:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(f'https://tmapi-alpha.transfermarkt.technology/player/{id}/market-value-history', headers=headers)
        if response.status_code == 200:
            try:
                values_raw = pd.json_normalize(response.json())
                values_exploded = pd.json_normalize(values_raw['data.history'].explode())
                final_player_values = values_exploded.merge(values_raw,left_on = 'playerId',right_on='data.current.playerId',how='left')
                final_player_values.to_csv('datasets/transfermarkt_players_values_2.csv',index=False,sep=',',mode='a')
                player_df_list.append(final_player_values)
            except:
                print(f"Failed to process data for player ID {id}")
                try_again.append(id)
        else:
            try_again.append(id)
        
        sleep_num = random.uniform(1.2,3.1)
        sleep(sleep_num)

    sleep_num = random.uniform(11.6,35.2)
    print(f"Batch {batch_counter} / {len(batch_players)} finished.")
    batch_counter += 1
    sleep(sleep_num)
    print(f"slept for {round(sleep_num,2)} seconds.")



allPlayerValues = pd.concat(player_df_list)
allPlayerValues.to_csv('datasets/transfermarkt_players_values_3.csv',index=False,sep=',',mode='w')

    


Batch 1 / 219 finished.
slept for 19.34 seconds.
Batch 2 / 219 finished.
slept for 24.84 seconds.
Batch 3 / 219 finished.
slept for 20.3 seconds.
Batch 4 / 219 finished.
slept for 18.64 seconds.
Batch 5 / 219 finished.
slept for 23.14 seconds.
Batch 6 / 219 finished.
slept for 30.06 seconds.
Batch 7 / 219 finished.
slept for 15.83 seconds.
Batch 8 / 219 finished.
slept for 26.18 seconds.
Batch 9 / 219 finished.
slept for 20.6 seconds.
Batch 10 / 219 finished.
slept for 22.64 seconds.
Batch 11 / 219 finished.
slept for 31.3 seconds.
Batch 12 / 219 finished.
slept for 27.81 seconds.
Batch 13 / 219 finished.
slept for 13.14 seconds.
Batch 14 / 219 finished.
slept for 19.86 seconds.
Batch 15 / 219 finished.
slept for 24.11 seconds.
Batch 16 / 219 finished.
slept for 29.37 seconds.
Batch 17 / 219 finished.
slept for 17.83 seconds.
Batch 18 / 219 finished.
slept for 32.75 seconds.
Batch 19 / 219 finished.
slept for 22.69 seconds.
Batch 20 / 219 finished.
slept for 13.91 seconds.
Batch 21 / 2

KeyError: 'data.current.playerId'

In [57]:
allPlayers = pd.read_csv('datasets/transfermarkt_players_values_2.csv',sep=',')
allPlayers.columns

Index(['playerId', 'clubId', 'age', 'marketValue.value',
       'marketValue.currency', 'marketValue.compact.prefix',
       'marketValue.compact.content', 'marketValue.compact.suffix',
       'marketValue.determined', 'success', 'message', 'data.history',
       'data.playerIds', 'data.clubIds', 'data.current.playerId',
       'data.current.clubId', 'data.current.age',
       'data.current.marketValue.value', 'data.current.marketValue.currency',
       'data.current.marketValue.compact.prefix',
       'data.current.marketValue.compact.content',
       'data.current.marketValue.compact.suffix',
       'data.current.marketValue.determined'],
      dtype='object')

In [64]:
player_ids2 = allPlayers['playerId'].unique().tolist()
player_ids3 = [x for x in player_ids2 if x not in player_ids]

print(len(player_ids3))

batch_players_retry = [player_ids3[x:x+30] for x in range(0, len(player_ids3), 30)]


player_df_list = []
try_again = []
batch_counter = 1
for b in batch_players_retry:
    for id in b:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(f'https://tmapi-alpha.transfermarkt.technology/player/{id}/market-value-history', headers=headers)
        if response.status_code == 200:
            try:
                values_raw = pd.json_normalize(response.json())
                values_exploded = pd.json_normalize(values_raw['data.history'].explode())
                final_player_values = values_exploded.merge(values_raw,left_on = 'playerId',right_on='data.current.playerId',how='left')
                final_player_values.to_csv('datasets/transfermarkt_players_values_2.csv',index=False,sep=',',mode='a')
                player_df_list.append(final_player_values)
            except:
                print(f"Failed to process data for player ID {id}")
                try_again.append(id)
        else:
            try_again.append(id)
        
        sleep_num = random.uniform(1.2,3.1)
        sleep(sleep_num)

    sleep_num = random.uniform(11.6,35.2)
    print(f"Batch {batch_counter} / {len(batch_players)} finished.")
    batch_counter += 1
    sleep(sleep_num)
    print(f"slept for {round(sleep_num,2)} seconds.")



allPlayerValues2 = pd.concat(player_df_list)
allPlayerValues.to_csv('datasets/transfermarkt_players_values_4.csv',index=False,sep=',',mode='w')


959
Batch 1 / 219 finished.
slept for 14.1 seconds.
Batch 2 / 219 finished.
slept for 15.99 seconds.
Batch 3 / 219 finished.
slept for 21.64 seconds.
Batch 4 / 219 finished.
slept for 34.43 seconds.
Batch 5 / 219 finished.
slept for 15.3 seconds.
Batch 6 / 219 finished.
slept for 31.5 seconds.
Batch 7 / 219 finished.
slept for 33.75 seconds.
Batch 8 / 219 finished.
slept for 22.59 seconds.
Batch 9 / 219 finished.
slept for 28.3 seconds.
Batch 10 / 219 finished.
slept for 17.35 seconds.
Batch 11 / 219 finished.
slept for 29.8 seconds.
Batch 12 / 219 finished.
slept for 20.58 seconds.
Batch 13 / 219 finished.
slept for 33.91 seconds.
Batch 14 / 219 finished.
slept for 27.21 seconds.
Batch 15 / 219 finished.
slept for 11.97 seconds.
Batch 16 / 219 finished.
slept for 17.48 seconds.
Batch 17 / 219 finished.
slept for 21.1 seconds.
Batch 18 / 219 finished.
slept for 14.9 seconds.
Batch 19 / 219 finished.
slept for 34.93 seconds.
Batch 20 / 219 finished.
slept for 20.32 seconds.
Batch 21 / 2

NameError: name 'allPlayerValues' is not defined

In [ ]:
for p in try_again:
    

In [65]:
final_player_values

,playerId,clubId,age,marketValue.value,marketValue.currency,marketValue.compact.prefix,marketValue.compact.content,marketValue.compact.suffix,marketValue.determined,success,...,data.clubIds,data.current.playerId,data.current.clubId,data.current.age,data.current.marketValue.value,data.current.marketValue.currency,data.current.marketValue.compact.prefix,data.current.marketValue.compact.content,data.current.marketValue.compact.suffix,data.current.marketValue.determined
0,800948,10958,18,1500000,EUR,€,1.50,M,2023-12-20,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
1,800948,10958,18,5000000,EUR,€,5.00,M,2024-03-12,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
2,800948,10958,19,5000000,EUR,€,5.00,M,2024-06-25,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
3,800948,58,19,7000000,EUR,€,7.00,M,2024-12-11,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
4,800948,58,20,8000000,EUR,€,8.00,M,2025-06-12,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
5,800948,10958,18,1500000,EUR,€,1.50,M,2023-12-20,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
6,800948,10958,18,5000000,EUR,€,5.00,M,2024-03-12,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
7,800948,10958,19,5000000,EUR,€,5.00,M,2024-06-25,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
8,800948,58,19,7000000,EUR,€,7.00,M,2024-12-11,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
9,800948,58,20,8000000,EUR,€,8.00,M,2025-06-12,True,...,"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12


In [30]:
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get('https://tmapi-alpha.transfermarkt.technology/player/800948/market-value-history', headers=headers)

response.json()

{'success': True,
 'message': 'OK',
 'data': {'history': [{'playerId': '800948',
    'clubId': '10958',
    'age': 18,
    'marketValue': {'value': 1500000,
     'currency': 'EUR',
     'compact': {'prefix': '€', 'content': '1.50', 'suffix': 'M'},
     'determined': '2023-12-20'}},
   {'playerId': '800948',
    'clubId': '10958',
    'age': 18,
    'marketValue': {'value': 5000000,
     'currency': 'EUR',
     'compact': {'prefix': '€', 'content': '5.00', 'suffix': 'M'},
     'determined': '2024-03-12'}},
   {'playerId': '800948',
    'clubId': '10958',
    'age': 19,
    'marketValue': {'value': 5000000,
     'currency': 'EUR',
     'compact': {'prefix': '€', 'content': '5.00', 'suffix': 'M'},
     'determined': '2024-06-25'}},
   {'playerId': '800948',
    'clubId': '58',
    'age': 19,
    'marketValue': {'value': 7000000,
     'currency': 'EUR',
     'compact': {'prefix': '€', 'content': '7.00', 'suffix': 'M'},
     'determined': '2024-12-11'}},
   {'playerId': '800948',
    'clubI

In [31]:
pd.json_normalize(response.json())

,success,message,data.history,data.playerIds,data.clubIds,data.current.playerId,data.current.clubId,data.current.age,data.current.marketValue.value,data.current.marketValue.currency,data.current.marketValue.compact.prefix,data.current.marketValue.compact.content,data.current.marketValue.compact.suffix,data.current.marketValue.determined
0,True,OK,"[{'playerId': '800948', 'clubId': '10958', 'ag...",[800948],"[10958, 58]",800948,58,20,8000000,EUR,€,8.00,M,2025-06-12
